In [42]:
import dimod
import numpy
import os
import json
from dimod.serialization import coo

from dwave.system import DWaveSampler
import minorminer as mm
from dwave.embedding import embed_bqm

qpu = DWaveSampler(solver="Advantage_system6.4")
qpu_graph = qpu.to_networkx_graph()

path_to_ocean_instances = "../data/instances/pruned/1"
path_to_velox_instances = "../velox/instances/pruned/1"
instance_list = os.listdir(path_to_ocean_instances)
instance_list = sorted(instance_list, key=lambda x: (int(x.split("_")[1]), int(x[:-5].split("_")[3])))
print("Number of instances: ", len(instance_list))
print(instance_list)
  

Number of instances:  12
['precision_2_timepoints_3.json', 'precision_2_timepoints_4.json', 'precision_2_timepoints_5.json', 'precision_2_timepoints_6.json', 'precision_3_timepoints_3.json', 'precision_3_timepoints_4.json', 'precision_3_timepoints_5.json', 'precision_3_timepoints_6.json', 'precision_4_timepoints_3.json', 'precision_4_timepoints_4.json', 'precision_4_timepoints_5.json', 'precision_4_timepoints_6.json']


In [51]:
import dimod.serialization

energies = []

for instance in instance_list:
  old_path = os.path.join(path_to_ocean_instances, instance)
  with open(old_path, 'r') as f:
    bqm = dimod.BinaryQuadraticModel.from_serializable(json.load(f))
  num_interactions = len(bqm.quadratic)
  num_variables = len(bqm.variables)
  density = num_interactions / (num_variables * (num_variables - 1) / 2)
  print(f"Density of instance {instance}: {density}")

  var_map = {i:i+1 for i in range(num_variables)}
  bqm.relabel_variables(var_map)
  bqm.change_vartype(dimod.SPIN, inplace=True)
  print(bqm.offset)
  energies.append(-bqm.offset)
  new_path = os.path.join(path_to_velox_instances, instance.replace(".json", ".coo"))
  with open(new_path, 'w') as f:
    f.write(f"# E0 = {-bqm.offset}\n")
    coo.dump(bqm, f)

  embedding = mm.find_embedding(dimod.to_networkx_graph(bqm), qpu_graph)
  bqm_embedded = embed_bqm(bqm, embedding, qpu_graph)
  bqm_embedded.relabel_variables_as_integers()
  new_var_map = {i:i+1 for i in range(len(bqm_embedded.variables))}
  bqm_embedded.relabel_variables(new_var_map)
  new_emb_path = os.path.join(path_to_velox_instances, instance.replace(".json", "_embedded.coo"))
  with open(new_emb_path, 'w') as f:
    f.write(f"# E0 = {-bqm_embedded.offset}\n")
    coo.dump(bqm_embedded, f)

Density of instance precision_2_timepoints_3.json: 0.21739130434782608
76.0
Density of instance precision_2_timepoints_4.json: 0.1935483870967742
110.0
Density of instance precision_2_timepoints_5.json: 0.16923076923076924
144.0
Density of instance precision_2_timepoints_6.json: 0.14893617021276595
178.0
Density of instance precision_3_timepoints_3.json: 0.22857142857142856
92.5
Density of instance precision_3_timepoints_4.json: 0.20212765957446807
133.0
Density of instance precision_3_timepoints_5.json: 0.17627118644067796
173.5
Density of instance precision_3_timepoints_6.json: 0.15492957746478872
214.0
Density of instance precision_4_timepoints_3.json: 0.23404255319148937
102.0
Density of instance precision_4_timepoints_4.json: 0.20634920634920634
146.125
Density of instance precision_4_timepoints_5.json: 0.17974683544303796
190.25
Density of instance precision_4_timepoints_6.json: 0.15789473684210525
234.375
